<a href="https://colab.research.google.com/github/adidonner/Machine-Deep-Leap-learning-Python-Based/blob/main/Task7_Cat_vs_Dogs_VGG_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import glob
import cv2
import random

In [7]:
directory = '/content/drive/MyDrive/cats_vs_dogs_data'

train_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    batch_size=32,
    image_size=(150, 150),
    shuffle=True,
    seed = 5,
    validation_split=0.2,
    subset='training',
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory,
    labels='inferred',
    batch_size=32,
    image_size=(150, 150),
    shuffle=False,
    seed = 5,
    validation_split=0.2,
    subset='validation',
)

Found 1467 files belonging to 2 classes.
Using 1174 files for training.
Found 1467 files belonging to 2 classes.
Using 293 files for validation.


# Without VGG

In [8]:
# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_ds, epochs=10, validation_data=val_ds)

# Plot sample images from each class and display the number of examples
# Code to plot images and display class distribution

Epoch 1/10
37/37 [==============================] - 117s 2s/step - loss: 59.5292 - accuracy: 0.9089 - val_loss: 0.6131 - val_accuracy: 0.8703
Epoch 2/10
37/37 [==============================] - 2s 58ms/step - loss: 0.3382 - accuracy: 0.9625 - val_loss: 0.2772 - val_accuracy: 0.9249
Epoch 3/10
37/37 [==============================] - 2s 56ms/step - loss: 0.2007 - accuracy: 0.9770 - val_loss: 0.2506 - val_accuracy: 0.9454
Epoch 4/10
37/37 [==============================] - 2s 56ms/step - loss: 0.1833 - accuracy: 0.9761 - val_loss: 0.1695 - val_accuracy: 0.9625
Epoch 5/10
37/37 [==============================] - 4s 90ms/step - loss: 0.0450 - accuracy: 0.9923 - val_loss: 0.4469 - val_accuracy: 0.9352
Epoch 6/10
37/37 [==============================] - 2s 57ms/step - loss: 0.0577 - accuracy: 0.9864 - val_loss: 0.2662 - val_accuracy: 0.9420
Epoch 7/10
37/37 [==============================] - 2s 57ms/step - loss: 0.0865 - accuracy: 0.9855 - val_loss: 0.1855 - val_accuracy: 0.9693
Epoch 8/10
3

# With VGG in Keras Architecture

Take this code (above "without VGG") and add to it: Freeze the first 3 layers of the VGG model except for FULLY CONNECTED and the last one. Use GPU and architecture - KERAS. Save the network weights

In [13]:
from keras.applications import VGG16
from keras.models import Model

# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the first 3 layers except for fully connected layers and the last one
for layer in base_model.layers[:3]:
    layer.trainable = False

# Create a new model by adding the top layers to the modified VGG16 base
x = base_model.output
x = Flatten()(x)
x = Dense(8, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_ds, epochs=10, validation_data=val_ds)

# Save the weights
model.save_weights('/content/drive/MyDrive/vgg16_frozen_weights_keras.h5')


Epoch 1/10
37/37 [==============================] - 11s 181ms/step - loss: 14.0905 - accuracy: 0.9199 - val_loss: 1.0662 - val_accuracy: 0.8396
Epoch 2/10
37/37 [==============================] - 7s 179ms/step - loss: 1.0366 - accuracy: 0.9651 - val_loss: 1.0303 - val_accuracy: 0.8396
Epoch 3/10
37/37 [==============================] - 7s 174ms/step - loss: 0.9925 - accuracy: 0.9651 - val_loss: 0.9963 - val_accuracy: 0.8396
Epoch 4/10
37/37 [==============================] - 7s 180ms/step - loss: 0.9506 - accuracy: 0.9651 - val_loss: 0.9641 - val_accuracy: 0.8396
Epoch 5/10
37/37 [==============================] - 7s 177ms/step - loss: 0.9105 - accuracy: 0.9651 - val_loss: 0.9337 - val_accuracy: 0.8396
Epoch 6/10
37/37 [==============================] - 7s 187ms/step - loss: 0.8723 - accuracy: 0.9651 - val_loss: 0.9050 - val_accuracy: 0.8396
Epoch 7/10
37/37 [==============================] - 7s 179ms/step - loss: 0.8358 - accuracy: 0.9651 - val_loss: 0.8778 - val_accuracy: 0.8396
Epoc